In [1]:
#!pip install fpdf

In [ ]:
import requests
import json
import os
from fpdf import FPDF

# Ollama API configuration for chat
OLLAMA_URL = "http://localhost:11434/api/chat"
MODEL_NAME = "phi3"

# Directory to save generated content
OUTPUT_DIR = "book_content"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def generate_toc(subject):
    """Generate a Table of Contents for the given subject using chat API."""
    prompt = f"""
    You are an expert assistant skilled at creating book outlines.
    Create a detailed Table of Contents for a book on the following subject:
    Subject: {subject}
    
    Structure the response as a valid JSON object with chapters and sections:
    {{
        "Table_of_Contents": [
            {{
                "Chapter 1": "Title of Chapter",
                "Sections": [
                    "Section 1.1: Title of Section",
                    "Section 1.2: Title of Section"
                ]
            }},
            {{
                "Chapter 2": "Title of Chapter",
                "Sections": [
                    "Section 2.1: Title of Section",
                    "Section 2.2: Title of Section"
                ]
            }}
        ]
    }}
    """
    
    # Chat API expects a list of messages with roles like "system", "user", and "assistant"
    messages = [
        {"role": "system", "content": "You are an expert book creator."},
        {"role": "user", "content": prompt}
    ]
    
    # Sending the request with a high timeout (e.g., 120 seconds)
    payload = {"model": MODEL_NAME, "messages": messages}
    try:
        response = requests.post(OLLAMA_URL, json=payload, timeout=120)
        if response.status_code != 200:
            raise Exception(f"Error from Ollama: {response.text}")

        content = response.json().get("response", "")
        try:
            toc = json.loads(content)
            return toc
        except json.JSONDecodeError:
            raise Exception(f"Failed to parse ToC JSON: {content}")
    except requests.exceptions.RequestException as e:
        raise Exception(f"Request failed: {str(e)}")

# Example usage
subject = "Prompt Engineering"
toc = generate_toc(subject)
print("Table of Contents generated:", toc)
